## Load Packages

In [1]:
using Distributions
#using Dates       #Julia 0.7

include("printmat.jl")

printmatDate (generic function with 8 methods)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Long-Run Returns as a Sum of Short-Run Returns

Let $R_{t}$ be a one-period net return. 
It is sometimes convenient to approximate the $q$-period return $Z_{q}$ as

$\begin{align}
Z_{q}  &  =(R_{1}+1)(R_{2}+1)\ldots(R_{q}+1)-1\nonumber\\
&  \approx R_{1}+R_{2}+\ldots+R_{q} 
\end{align}$

Log returns $r=\ln (1+R)$ are better suited for multi-period analysis since there is no approximation error in

$z_{q}=r_{1}+r_{2}+\ldots+r_{q}$

In [3]:
R_A = [5;-5;5]/100            #net returns portfolio A in three periods
R_B = [20;-35;25]/100

r_A = log.(1 .+ R_A)             #log returns
r_B = log.(1 .+ R_B)   

Price_A = cumprod(1 .+ R_A)     #price of portfolio A, assuming price in t=0 is 1.0
Price_B = cumprod(1 .+ R_B)

println("time      Price_A      Price_B")
printmat([(1:3) Price_A Price_B])

println("Average net returns: A and B, in %")
printmat(mean([R_A R_B],1)*100)

println("Average log returns: A and B, in %")
printmat(mean([r_A r_B],1)*100)

time      Price_A      Price_B
     1.000     1.050     1.200
     2.000     0.997     0.780
     3.000     1.047     0.975

Average net returns: A and B, in %
     1.667     3.333

Average log returns: A and B, in %
     1.543    -0.844



# Distribution of Long-Run Returns in the iid Case

If $r$ is iid N$(\mu,\sigma^2)$, then $z_q$ is N$(q\mu,q\sigma^2)$. 

In [4]:
function ΦNS(x,μ=0,σ²=1)                          #cdf of N(μ,σ²), NS for "non-standard"
    Pr = cdf(Normal(μ,sqrt(σ²)),x)
  return Pr
end

function ϕNS(x,μ=0,σ²=1)              #pdf of N(μ,σ²), NS for "non-standard"
    pdfx = pdf(Normal(μ,sqrt(σ²)),x)
    return pdfx
end

ϕNS (generic function with 3 methods)

## Pdfs of Long-Run Returns (for Different Horizons)

In [5]:
Z = linspace(-50,100,101)
μ = 0.06*100
σ = 0.19*100
pdf_Z1y  = ϕNS.(Z,μ,σ^2)              #density functions of returns
pdf_Z10y = ϕNS.(Z,10*μ,10*σ^2)

println()

In [6]:
plot(Z,pdf_Z1y,color=:red,linewidth=2,label="1y")
plot!(Z,pdf_Z10y,color=:blue,linewidth=2,label="10y")
title!("pdf(long run excess returns)")
xlabel!("excess return, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -50 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 0.005 
 
 
 0.010 
 
 
 0.015 
 
 
 0.020 
 
 
 pdf(long run excess returns) 
 
 
 excess return, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 45.4014,355.797 50.7505,354.695 56.0996,353.351 61.4488,351.722 66.7979,349.763 72.147,347.424 77.4962,344.649 82.8453,341.382 88.1945,337.564 93.5436,333.134 
 98.8927,328.032 104.242,322.2 109.591,315.584 114.94,308.138 120.289,299.822 125.638,290.608 130.988,280.483 136.337,269.448 141.686,257.524 147.035,244.753 
 152.384,231.196 157.733,216.941 163.082,202.096 168.431,186.795 173.781,171.193 179.13,155.465 184.479,139.804 189.828,124.418 195.177,109.524 200.526,95.3434 
 205.875,82.0982 211.225,70.0037 216.574,59.2638 221.923,50.0643 227.272,42.5685 232.621,36.9119 237.97,33.1982 243.319,31.4961 248.669,31.8372 254.018,34.2153 
 259.367,38.586 264.716,44.8688 270.065,52.949 275.414,62.6814 280.763,73.8945 286.112,86.3958 291.462,99.9772 296.811,114.42 302.16,129.503 307.509,145.004 
 312.858,160.71 318.207,176.416 323.556,191.937 328.906,207.102 334.255,221.764 339.604,235.797 344.953,249.101 350.302,261.596 355.651,273.226 361,283.959 
 366.35,293.78 371.699,302.692 377.048,310.715 382.397,317.88 387.746,324.228 393.095,329.811 398.444,334.682 403.793,338.902 409.143,342.529 414.492,345.626 
 419.841,348.249 425.19,350.456 430.539,352.3 435.888,353.828 441.237,355.087 446.587,356.117 451.936,356.953 457.285,357.628 462.634,358.168 467.983,358.598 
 473.332,358.937 478.681,359.204 484.031,359.412 489.38,359.572 494.729,359.696 500.078,359.791 505.427,359.862 510.776,359.916 516.125,359.957 521.474,359.987 
 526.824,360.009 532.173,360.025 537.522,360.037 542.871,360.046 548.22,360.052 553.569,360.057 558.918,360.06 564.268,360.062 569.617,360.064 574.966,360.065 
 580.315,360.065 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:2; stroke-opacity:1; fill:none" points="
 45.4014,340.616 50.7505,339.713 56.0996,338.781 61.4488,337.82 66.7979,336.83 72.147,335.811 77.4962,334.764 82.8453,333.688 88.1945,332.583 93.5436,331.449 
 98.8927,330.288 104.242,329.099 109.591,327.882 114.94,326.638 120.289,325.368 125.638,324.072 130.988,322.751 136.337,321.405 141.686,320.036 147.035,318.644 
 152.384,317.231 157.733,315.797 163.082,314.343 168.431,312.872 173.781,311.383 179.13,309.878 184.479,308.359 189.828,306.828 195.177,305.285 200.526,303.733 
 205.875,302.172 211.225,300.606 216.574,299.035 221.923,297.462 227.272,295.888 232.621,294.316 237.97,292.747 243.319,291.184 248.669,289.628 254.018,288.082 
 259.367,286.548 264.716,285.028 270.065,283.524 275.414,282.039 280.763,280.575 286.112,279.133 291.462,277.717 296.811,276.328 302.16,274.969 307.509,273.641 
 312.858,272.348 318.207,271.091 323.556,269.872 328.906,268.693 334.255,267.556 339.604,266.464 344.953,265.418 350.302,264.42 355.651,263.471 361,262.574 
 366.35,261.73 371.699,260.94 377.048,260.206 382.397,259.53 387.746,258.911 393.095,258.353 398.444,257.855 403.793,257.419 409.143,257.045 414.492,256.734 
 419.841,256.487 425.19,256.304 430.539,256.185 435.888,256.131 441.237,256.142 446.587,256.218 451.936,256.358 457.285,256.562 462.634,256.831 467.983,257.163 
 473.332,257.557 478.681,258.014 484.031,258.532 489.38,259.111 494.729,259.749 500.078,260.444 505.427,261.197 510.776,262.005 516.125,262.867 521.474,263.782 
 526.824,264.747 532.173,265.761 537.522,266.823 542.871,267.93 548.22,269.081 553.569,270.274 558.918,271.506 564.268,272.775 569.617,274.08 574.966,275.418 
 580.315,276.788 
 "/>
 
 
 
 
 1y 
 
 
 
 10y

##    Prob(Zᵉ < 0) for Different Horizons

In [7]:
t = 1:10

ProbNegReturn = fill(NaN,length(t))
for i in t
    ProbNegReturn[i] = ΦNS(0,i*μ,i*σ^2)    #cdf(0) for different horizons
end

In [8]:
plot(t,ProbNegReturn*100,color=:red,linewidth=2,legend=false)
title!("Pr(Ze<0)")
xlabel!("Investment horizon (years)")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 20 
 
 
 25 
 
 
 30 
 
 
 35 
 
 
 Pr(Ze<0) 
 
 
 Investment horizon (years) 
 
 
 %

## E(Zᵉ|Zᵉ < 0) for Different Horizons

In [9]:
t = 1:10

CondER = fill(NaN,length(t))
for i in t
    CondER[i] = i*μ - sqrt(i)*σ*ϕNS(-sqrt(i)*μ/σ)/ΦNS(-sqrt(i)*μ/σ)  
end

In [10]:
plot(t,CondER,color=:red,linewidth=2,legend=false)
title!("E(Ze|Ze<0)")
xlabel!("Investment horizon (years)")
ylabel!("%")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
 
 10 
 
 
 -30 
 
 
 -25 
 
 
 -20 
 
 
 -15 
 
 
 E(Ze|Ze<0) 
 
 
 Investment horizon (years) 
 
 
 %